In [8]:
file_path = "abfss://fsogvd@datalakeogvdlabs.dfs.core.windows.net/user/NYCTripSmall.parquet"

df_taxi = spark.read.parquet(file_path)

print("Datos NYC Taxi cargados desde Data Lake:")
print(f"Total registros: {df_taxi.count()}")
print(f"Total columnas: {len(df_taxi.columns)}")
print("\nPrimeras 5 filas:")
df_taxi.show(5)

print("\nEsquema de datos:")
df_taxi.printSchema()

StatementMeta(sparkpool01, 0, 8, Finished, Available, Finished)

Datos NYC Taxi cargados desde Data Lake:
Total registros: 68211
Total columnas: 20

Primeras 5 filas:
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2023-01-01 00:26:10|  2023-01-01 00:37:11|        

In [9]:
df_taxi.createOrReplaceTempView("nyc_taxi")
print("Vista temporal 'nyc_taxi' creada en base de datos Spark")

row_count = spark.sql("SELECT COUNT(*) as total FROM nyc_taxi").collect()[0]['total']
print(f"Total de registros en vista: {row_count}")

StatementMeta(sparkpool01, 0, 9, Finished, Available, Finished)

Vista temporal 'nyc_taxi' creada en base de datos Spark
Total de registros en vista: 68211


In [10]:
print("ANÁLISIS 1: Estadísticas básicas")
basic_stats = spark.sql("""
SELECT 
    COUNT(*) as total_trips,
    AVG(total_amount) as avg_fare,
    MIN(total_amount) as min_fare,
    MAX(total_amount) as max_fare,
    AVG(trip_distance) as avg_distance,
    MIN(trip_distance) as min_distance,
    MAX(trip_distance) as max_distance
FROM nyc_taxi
WHERE total_amount > 0 AND trip_distance > 0
""")
basic_stats.show()

print("\nANÁLISIS 2: Distribución por tipo de pago")
payment_analysis = spark.sql("""
SELECT 
    payment_type,
    COUNT(*) as trip_count,
    ROUND(AVG(total_amount), 2) as avg_amount,
    ROUND(SUM(total_amount), 2) as total_revenue,
    ROUND(AVG(trip_distance), 2) as avg_distance
FROM nyc_taxi 
WHERE total_amount > 0
GROUP BY payment_type
ORDER BY trip_count DESC
""")
payment_analysis.show()

print("\nANÁLISIS 3: Patrones temporales por hora")
temporal_analysis = spark.sql("""
SELECT 
    HOUR(lpep_pickup_datetime) as pickup_hour,
    COUNT(*) as trips_count,
    ROUND(AVG(trip_distance), 2) as avg_distance,
    ROUND(AVG(total_amount), 2) as avg_fare
FROM nyc_taxi
WHERE lpep_pickup_datetime IS NOT NULL
GROUP BY HOUR(lpep_pickup_datetime)
ORDER BY pickup_hour
""")
temporal_analysis.show(24)

StatementMeta(sparkpool01, 0, 10, Finished, Available, Finished)

ANÁLISIS 1: Estadísticas básicas
+-----------+------------------+--------+--------+-----------------+------------+------------+
|total_trips|          avg_fare|min_fare|max_fare|     avg_distance|min_distance|max_distance|
+-----------+------------------+--------+--------+-----------------+------------+------------+
|      64742|21.767631985419392|    1.01|   456.0|8.547262982298957|        0.01|   120098.84|
+-----------+------------------+--------+--------+-----------------+------------+------------+


ANÁLISIS 2: Distribución por tipo de pago
+------------+----------+----------+-------------+------------+
|payment_type|trip_count|avg_amount|total_revenue|avg_distance|
+------------+----------+----------+-------------+------------+
|         1.0|     40645|     23.03|    936029.08|        2.62|
|         2.0|     22605|      18.4|    415884.75|        2.54|
|        null|      4319|     30.53|    131879.51|       90.03|
|         3.0|       330|     10.43|      3442.35|        0.92|


In [11]:
print("ANÁLISIS 4: Top ubicaciones de pickup")
pickup_locations = spark.sql("""
SELECT 
    PULocationID,
    COUNT(*) as pickup_count,
    ROUND(AVG(total_amount), 2) as avg_fare,
    ROUND(AVG(trip_distance), 2) as avg_distance
FROM nyc_taxi
WHERE PULocationID IS NOT NULL
GROUP BY PULocationID
ORDER BY pickup_count DESC
LIMIT 15
""")
pickup_locations.show()

print("\nANÁLISIS 5: Distribución por día de la semana")
weekday_analysis = spark.sql("""
SELECT 
    DAYOFWEEK(lpep_pickup_datetime) as day_of_week,
    CASE DAYOFWEEK(lpep_pickup_datetime)
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END as day_name,
    COUNT(*) as trips_count,
    ROUND(AVG(total_amount), 2) as avg_fare
FROM nyc_taxi
WHERE lpep_pickup_datetime IS NOT NULL
GROUP BY DAYOFWEEK(lpep_pickup_datetime)
ORDER BY day_of_week
""")
weekday_analysis.show()

print("\nGuardando resultados de análisis...")
payment_analysis.write.mode("overwrite").parquet("abfss://fsogvd@datalakeogvdlabs.dfs.core.windows.net/user/payment_analysis.parquet")
temporal_analysis.write.mode("overwrite").parquet("abfss://fsogvd@datalakeogvdlabs.dfs.core.windows.net/user/temporal_analysis.parquet")
print("Análisis guardados en Data Lake")

StatementMeta(sparkpool01, 0, 11, Finished, Available, Finished)

ANÁLISIS 4: Top ubicaciones de pickup
+------------+------------+--------+------------+
|PULocationID|pickup_count|avg_fare|avg_distance|
+------------+------------+--------+------------+
|          74|       13257|   19.34|        2.64|
|          75|        9109|   20.11|        2.29|
|          41|        4058|   18.16|       14.42|
|         166|        3879|   21.57|        2.56|
|          95|        3839|   19.13|        2.44|
|          82|        2927|   21.35|         6.9|
|          43|        2868|   22.17|        2.38|
|          97|        2195|   21.61|        2.45|
|           7|        1965|   20.35|        3.56|
|         244|        1804|   31.31|       43.15|
|          42|        1800|   18.61|        3.69|
|          65|        1546|   24.68|        2.98|
|         260|        1409|    21.6|        2.82|
|         129|        1320|   21.09|        2.62|
|         130|        1238|   26.79|        4.14|
+------------+------------+--------+------------+


ANÁLISIS 5